# 투구정보 Data EDA & Processing Note

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.min_rows = 10
pd.options.display.max_rows = 20

In [ ]:
p_all = pd.read_csv("/content/drive/MyDrive/AI/중간프로젝트/KBO전략데이터.csv")

In [ ]:
p_all

,game_date,refree,home_name,away_name,inning,x_spot,y_spot,ball_speed,judgement,ball_type,...,pitcher_position,hitter_name,hitter_position,game_place,game_start_time,hitter_team,hitter_birth,hitter_height,hitter_weight,hitter_salary
0,2019-03-23,강광회,두산,한화,1,50.587819,112.311012,141.0,헛스윙,커터,...,R,김재환,L,잠실,14:00,두산,1988-09-22,183cm,90kg,150000만원
1,2019-03-23,강광회,두산,한화,1,50.587819,112.311012,141.0,헛스윙,커터,...,R,김재환,L,잠실,14:00,두산,1988-09-22,183cm,90kg,150000만원
2,2019-03-23,강광회,두산,한화,1,50.587819,112.311012,141.0,헛스윙,커터,...,R,김재환,L,잠실,14:00,두산,1988-09-22,183cm,90kg,150000만원
3,2019-03-23,강광회,두산,한화,1,6.315229,34.321115,144.0,볼,투심,...,R,김재환,L,잠실,14:00,두산,1988-09-22,183cm,90kg,150000만원
4,2019-03-23,강광회,두산,한화,1,6.315229,34.321115,144.0,볼,투심,...,R,김재환,L,잠실,14:00,두산,1988-09-22,183cm,90kg,150000만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2941116,2023-08-09,유덕형,키움,롯데,4,75.728521,175.900863,120.0,1루타,커브,...,R,도슨,L,고척,18:30,키움,1995-05-19,188cm,97kg,80000달러
2941117,2023-08-09,유덕형,키움,롯데,5,121.388820,184.385069,130.0,1루타,슬라이더,...,L,도슨,L,고척,18:30,키움,1995-05-19,188cm,97kg,80000달러
2941118,2023-08-09,유덕형,키움,롯데,7,47.502201,153.183378,143.0,스트라이크,직구,...,R,도슨,L,고척,18:30,키움,1995-05-19,188cm,97kg,80000달러
2941119,2023-08-09,유덕형,키움,롯데,7,105.033290,125.251858,143.0,파울,직구,...,R,도슨,L,고척,18:30,키움,1995-05-19,188cm,97kg,80000달러


In [ ]:
# 데이터 load 및 불필요한 피쳐 제거
df_szone = p_all[p_all.judgement=="스트라이크"].drop(["hitter_birth", "hitter_salary"], axis=1)
df_szone = df_szone.dropna()

In [ ]:
# 데이터 연속화
df_szone.hitter_height = df_szone.hitter_height.str.replace("cm", "")
df_szone.hitter_weight = df_szone.hitter_weight.str.replace("kg", "")
df_szone = df_szone.rename(columns = {"ball_speed":"ball_speed(km/h)", "hitter_height":"hitter_height(cm)", "hitter_weight":"hitter_weight(kg)"})

## OPS 예측을 위한 타자별 경기기록 분석

### 가설
- 최근 7경기의 타석수, 안타, 2루타, 3루타, 홈런, 볼넷, 사구 기록을 통해 타자의 OPS를 예측할 수 있다.
- 해당 피쳐들은 시간의 영향을 받으므로 예측 당일의 날짜를 추가 피쳐로 주면 유의미한 시계열 학습을 유도할 수 있다.

In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

import warnings
warnings.filterwarnings(action='ignore')

ModuleNotFoundError: ignored

In [ ]:
# H, 2B, 3B, HR, BB, HBP, AB 계산기
driver = webdriver.Chrome()

url = "https://www.koreabaseball.com/Record/Player/HitterDetail/Daily.aspx?playerId=77532"
driver.get(url)
player_name = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblName"]').text
daily = driver.find_element(By.XPATH, '//*[@id="contents"]/div[2]/div[1]/div[3]/ul/li[3]')
daily.click()

X_H = pd.DataFrame()
y_H = pd.DataFrame()
X_2B = pd.DataFrame()
y_2B = pd.DataFrame()
X_3B = pd.DataFrame()
y_3B = pd.DataFrame()
X_HR = pd.DataFrame()
y_HR = pd.DataFrame()
X_BB = pd.DataFrame()
y_BB = pd.DataFrame()
X_HBP = pd.DataFrame()
y_HBP = pd.DataFrame()
X_AB = pd.DataFrame()
y_AB = pd.DataFrame()

today_H = pd.DataFrame()
today_2B = pd.DataFrame()
today_3B = pd.DataFrame()
today_HR = pd.DataFrame()
today_BB = pd.DataFrame()
today_HBP = pd.DataFrame()
today_AB = pd.DataFrame()

# 데이터 전처리기
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

ohe = OneHotEncoder()
ohe_fitted = 0
ss_H = StandardScaler()
ss_2B = StandardScaler()
ss_3B = StandardScaler()
ss_HR = StandardScaler()
ss_BB = StandardScaler()
ss_HBP = StandardScaler()
ss_AB = StandardScaler()

for year in ["2023", "2022", "2021", "2020", "2019", "2018", "2017"]:
    base = pd.DataFrame()

    df_H = pd.DataFrame()
    df_2B = pd.DataFrame()
    df_3B = pd.DataFrame()
    df_HR = pd.DataFrame()
    df_BB = pd.DataFrame()
    df_HBP = pd.DataFrame()
    df_AB = pd.DataFrame()

    year_dropdown = Select(driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlYear"]'))
    year_dropdown.select_by_value(str(year))
    html = pd.read_html(driver.page_source)

    for i in range(len(html)):
        month = html[i].columns[0]
        opponent = html[i].columns[1]
        html[i].rename(columns={opponent:"opponent"}, inplace=True)

        table = html[i].drop(index=len(html[i])-1)
        table = table.drop(month, axis=1)

        # 결측치 제거
        table = table.replace("-", np.nan)
        table.dropna(inplace=True)

        base = pd.concat([base, table], axis=0)

    # 3경기를 하나의 인스턴스로 생성.
    try:
        for i in range(3, len(base)):
            nr_H, nr_2B, nr_3B, nr_HR, nr_BB, nr_HBP, nr_AB = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
            cnt = 3
            for x in base[i-3:i].iterrows():
                # y값이 될 피쳐 Drop
                H = x[1].drop("H")
                _2B = x[1].drop("2B")
                _3B = x[1].drop("3B")
                HR = x[1].drop("HR")
                BB = x[1].drop("BB")
                HBP = x[1].drop("HBP")
                AB = x[1].drop("AB")

                nr_H = pd.concat([nr_H, pd.DataFrame(H.values.reshape(1, -1), columns=H.index + "_" + str(cnt) + "ago")], axis=1)
                nr_2B = pd.concat([nr_2B, pd.DataFrame(_2B.values.reshape(1, -1), columns=_2B.index + "_" + str(cnt) + "ago")], axis=1)
                nr_3B = pd.concat([nr_3B, pd.DataFrame(_3B.values.reshape(1, -1), columns=_3B.index + "_" + str(cnt) + "ago")], axis=1)
                nr_HR = pd.concat([nr_HR, pd.DataFrame(HR.values.reshape(1, -1), columns=HR.index + "_" + str(cnt) + "ago")], axis=1)
                nr_BB = pd.concat([nr_BB, pd.DataFrame(BB.values.reshape(1, -1), columns=BB.index + "_" + str(cnt) + "ago")], axis=1)
                nr_HBP = pd.concat([nr_HBP, pd.DataFrame(HBP.values.reshape(1, -1), columns=HBP.index + "_" + str(cnt) + "ago")], axis=1)
                nr_AB = pd.concat([nr_AB, pd.DataFrame(AB.values.reshape(1, -1), columns=AB.index + "_" + str(cnt) + "ago")], axis=1)

                cnt -= 1

            else:
                y_H = {"y":[base.iloc[i]["H"]]}
                y_2B = {"y":[base.iloc[i]["2B"]]}
                y_3B = {"y":[base.iloc[i]["3B"]]}
                y_HR = {"y":[base.iloc[i]["HR"]]}
                y_BB = {"y":[base.iloc[i]["BB"]]}
                y_HBP = {"y":[base.iloc[i]["HBP"]]}
                y_AB = {"y":[base.iloc[i]["AB"]]}

                nr_H = pd.concat([nr_H, pd.DataFrame(y_H)], axis=1)
                nr_2B = pd.concat([nr_2B, pd.DataFrame(y_2B)], axis=1)
                nr_3B = pd.concat([nr_3B, pd.DataFrame(y_3B)], axis=1)
                nr_HR = pd.concat([nr_HR, pd.DataFrame(y_HR)], axis=1)
                nr_BB = pd.concat([nr_BB, pd.DataFrame(y_BB)], axis=1)
                nr_HBP = pd.concat([nr_HBP, pd.DataFrame(y_HBP)], axis=1)
                nr_AB = pd.concat([nr_AB, pd.DataFrame(y_AB)], axis=1)

            # 당일 데이터 추가
            if year == "2023" and i == len(base)-1:
                today_H = pd.concat([today_H, nr_H], axis=1).drop("y", axis=1)
                today_2B = pd.concat([today_2B, nr_2B], axis=1).drop("y", axis=1)
                today_3B = pd.concat([today_3B, nr_3B], axis=1).drop("y", axis=1)
                today_HR = pd.concat([today_HR, nr_HR], axis=1).drop("y", axis=1)
                today_BB = pd.concat([today_BB, nr_BB], axis=1).drop("y", axis=1)
                today_HBP = pd.concat([today_HBP, nr_HBP], axis=1).drop("y", axis=1)
                today_AB = pd.concat([today_AB, nr_AB], axis=1).drop("y", axis=1)

            df_H = pd.concat([df_H, nr_H], axis=0)
            df_2B = pd.concat([df_2B, nr_2B], axis=0)
            df_3B = pd.concat([df_3B, nr_3B], axis=0)
            df_HR = pd.concat([df_HR, nr_HR], axis=0)
            df_BB = pd.concat([df_BB, nr_BB], axis=0)
            df_HBP = pd.concat([df_HBP, nr_HBP], axis=0)
            df_AB = pd.concat([df_AB, nr_AB], axis=0)

    # 3경기 이하 선발된 연도의 데이터는 제외.
    except:
        continue

In [ ]:
df_H.to_csv("H분석-손아섭.csv")
df_H = df_H.reset_index(drop=True)

In [ ]:
df_H.groupby([i for i in df_H.columns if i != "y"]).mean()

y
opponent_3ago AVG1_3ago PA_3ago AB_3ago R_3ago 2B_3ago 3B_3ago HR_3ago RBI_3ago SB_3ago CS_3ago BB_3ago HBP_3ago SO_3ago GDP_3ago AVG2_3ago opponent_2ago AVG1_2ago PA_2ago AB_2ago R_2ago 2B_2ago 3B_2ago HR_2ago RBI_2ago SB_2ago CS_2ago BB_2ago HBP_2ago SO_2ago GDP_2ago AVG2_2ago opponent_1ago AVG1_1ago PA_1ago AB_1ago R_1ago 2B_1ago 3B_1ago HR_1ago RBI_1ago SB_1ago CS_1ago BB_1ago HBP_1ago SO_1ago GDP_1ago AVG2_1ago     
KIA           0.000     4       4       0      0       0       0       0        0       0       0       0        2       0        0.337     KIA           0.250     4       4       0      0       0       0       0        0       0       0       0        3       0        0.336     한화            0.800     5       5       1      0       0       1       4        0       0       0       0        1       0        0.342      1.0
              0.200     5       5       0      0       0       0       0        0       0       0       0        1       0        0.333     KIA           0.750     5       4       2      1       0       0       1        1       0       1       0        0       0        0.336     LG            0.600     5       5       2      0       0       1       3        2       0       0       0        0       0        0.339      2.0
              0.250     4       4       0      0       0       0       0        0       0       0       0        0       0        0.341     KIA           0.250     4       4       0      1       0       0       0        0       0       0       0        1       0        0.340     SK            0.333     4       3       0      0       0       0       0        0       0       1       0        0       0        0.340      0.0
                                                                                                                         1        0.326     KIA           0.333     6       6       1      0       0       0       0        0       0       0       0        3       0        0.326     삼성            0.000     4       4       0      0       0       0       0        0       0       0       0        0       0        0.320      1.0
                                                                                                                 2       0        0.283     KIA           0.600     5       5       2      1       0       1       1        0       0       0       0        1       0        0.296     한화            0.667     6       6       3      1       0       1       3        1       1       0       0        2       0        0.313      1.0
...                                                                                                                                                                                                                                                                                                                                                                                                                                  ...
한화            0.500     4       2       2      0       0       0       0        1       0       2       0        0       0        0.340     SK            0.000     4       4       0      0       0       0       0        0       0       0       0        0       0        0.337     삼성            0.750     5       4       1      0       0       0       0        0       0       1       0        0       0        0.340      2.0
                        5       4       0      0       0       0       0        0       0       1       0        0       0        0.336     한화            0.500     5       4       0      1       0       0       1        0       0       0       0        1       0        0.338     삼성            0.250     4       4       1      0       0       1       2        0       0       0       0        2       0        0.337      2.0
                                               1       0       0       1        0       0       0       0        1       0        0.338     삼성            0.250     4       4

In [ ]:
df_HR.dropna(inplace=True)
y = df_HR["y"]

X_cate = df_H[[f"opponent_{i}ago" for i in range(3, 0, -1)]]
X_conti = df_H[[i for i in df_H.columns if i not in X_cate.columns]].drop("y", axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

ohe = OneHotEncoder()
data = ohe.fit_transform(X_cate).toarray()
cols = ohe.get_feature_names_out()
X_cate = pd.DataFrame(data, columns=cols)

ss = StandardScaler()
data = ss.fit_transform(X_conti)
cols = ss.get_feature_names_out()
X_conti = pd.DataFrame(data, columns=cols)

s_X = pd.concat([X_cate,X_conti], axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

mm = KNeighborsClassifier()
m = BaggingClassifier(estimator=mm)

print(cross_val_score(m, s_X, y))

[0.86206897 0.89285714 0.85714286 0.85714286 0.85714286]


## 투수별 로케이션 분석

### 데이터를 통해 확인할 수 있는 가설  
- 상대 타자의 포지션에 따라 투수가 던지는 공의 평균 좌표와 최빈 구종이 다르다.
- 상대 타자가 누군지에 따라 투수가 던지는 공의 평균 좌표와 최빈 구종이 다르다.
- 이닝마다 투수가 던지는 공의 평균 좌표와 최빈 구종이 다르다.

#### 상대 타자의 포지션에 따라 투수가 던지는 공의 평균 좌표와 최빈 구종

In [ ]:
# 상대 타자의 포지션에 따라 투수가 던지는 공의 평균 좌표와 최빈 구종
print(f"""x좌표
{df_szone.groupby(["pitcher_name", "hitter_position"]).x_spot.median()}

y좌표
{df_szone.groupby(["pitcher_name", "hitter_position"]).y_spot.median()}

구종
{df_szone.groupby(["pitcher_name", "hitter_position", "ball_type"]).ball_type.count()}""")

x좌표
pitcher_name  hitter_position
가뇽            L                  111.280597
              R                   96.868249
가빌리오          L                  110.529134
              R                   91.097051
강경학           L                   60.221140
                                    ...    
황인준           R                   77.758586
후라도           L                   96.497956
              R                   87.373510
후랭코프          L                  111.615707
              R                   87.490842
Name: x_spot, Length: 1038, dtype: float64

y좌표
pitcher_name  hitter_position
가뇽            L                  143.038463
              R                  151.166555
가빌리오          L                  148.002349
              R                  166.475126
강경학           L                   92.849366
                                    ...    
황인준           R                  165.571812
후라도           L                  141.018166
              R                  153.011400
후랭코프    

#### 상대 타자가 누군지에 따라 투수가 던지는 공의 평균 좌표와 최빈 구종

In [ ]:
# 상대 타자가 누군지에 따라 달라지는 투수 로케이션
print(f"""x좌표
{df_szone.groupby(["pitcher_name", "hitter_name"]).x_spot.median()}

y좌표
{df_szone.groupby(["pitcher_name", "hitter_name"]).y_spot.median()}

구종
{df_szone.groupby(["pitcher_name", "hitter_name", "ball_type"]).ball_type.count()}""")

x좌표
pitcher_name  hitter_name
가뇽            강민호             70.047061
              강백호            149.567200
              강진성            100.131089
              강한울             94.923791
              구자욱            126.065421
                                ...    
후랭코프          채은성             85.968680
              최정             103.584481
              최항             141.515932
              최형우            129.580752
              허도환            145.799277
Name: x_spot, Length: 34680, dtype: float64

y좌표
pitcher_name  hitter_name
가뇽            강민호            178.745000
              강백호            116.813686
              강진성            160.944063
              강한울            129.675058
              구자욱            153.051789
                                ...    
후랭코프          채은성            118.862646
              최정             139.270600
              최항             188.646280
              최형우             86.575240
              허도환             74.054783
Name: y_spot, L

#### 투수가 던지는 공의 이닝별 평균 좌표와 최빈 구종

In [ ]:
# 투수가 던지는 공의 이닝별 평균 좌표와 최빈 구종
print(f"""x좌표
{df_szone.groupby(["pitcher_name", "inning"]).x_spot.median()}

y좌표
{df_szone.groupby(["pitcher_name", "inning"]).y_spot.median()}

구종
{df_szone.groupby(["pitcher_name", "inning", "ball_type"]).ball_type.count()}""")

x좌표
pitcher_name  inning
가뇽            1         102.474879
              2         110.044967
              3         115.478503
              4         102.545177
              5          98.616615
                           ...    
후랭코프          3         104.265713
              4          95.179136
              5         103.584481
              6          99.453114
              7         108.881784
Name: x_spot, Length: 3765, dtype: float64

y좌표
pitcher_name  inning
가뇽            1         135.913138
              2         142.924680
              3         151.135286
              4         151.166555
              5         170.594326
                           ...    
후랭코프          3         140.076635
              4         142.416566
              5         139.270600
              6         130.829823
              7         108.477138
Name: y_spot, Length: 3765, dtype: float64

구종
pitcher_name  inning  ball_type
가뇽            1       슬라이더          46
                  

#### 상황에 따른 투수별 투구 로케이션

In [ ]:
# 상황에 따른 투수별 투구 로케이션
print(f"""x좌표
{df_szone.groupby(["pitcher_name", "hitter_name", "inning"]).x_spot.mean()}

 y좌표
{df_szone.groupby(["pitcher_name", "hitter_name", "inning"]).y_spot.mean()}

구종
{df_szone.groupby(["pitcher_name", "hitter_name", "inning", "ball_type"]).ball_type.count()}""")

x좌표
pitcher_name  hitter_name  inning
가뇽            강민호          3          83.535087
                           4          56.559034
              강백호          1         108.569906
                           2         172.053343
                           4         149.567200
                                        ...    
후랭코프          최항           3         171.416157
                           4         111.615707
              최형우          4          82.171147
                           6         176.990357
              허도환          4         145.799277
Name: x_spot, Length: 73557, dtype: float64

 y좌표
pitcher_name  hitter_name  inning
가뇽            강민호          3         189.895286
                           4         167.594715
              강백호          1         170.479240
                           2          84.316955
                           4          97.241298
                                        ...    
후랭코프          최항           3         205.085732
              

#### 머신러닝을 적용하기는 부족한 데이터 인스턴스

In [ ]:
# 머신러닝을 적용하기는 부족한 데이터 인스턴스
print(f"""x좌표
{df_szone.groupby(["pitcher_name", "hitter_name", "inning"]).x_spot.count()}

 y좌표
{df_szone.groupby(["pitcher_name", "hitter_name", "inning"]).y_spot.count()}

구종
{df_szone.groupby(["pitcher_name", "hitter_name", "inning", "ball_type"]).ball_type.count()}""")

x좌표
pitcher_name  hitter_name  inning
가뇽            강민호          3         7
                           4         7
              강백호          1         3
                           2         3
                           4         6
                                    ..
후랭코프          최항           3         3
                           4         3
              최형우          4         7
                           6         7
              허도환          4         3
Name: x_spot, Length: 73557, dtype: int64

 y좌표
pitcher_name  hitter_name  inning
가뇽            강민호          3         7
                           4         7
              강백호          1         3
                           2         3
                           4         6
                                    ..
후랭코프          최항           3         3
                           4         3
              최형우          4         7
                           6         7
              허도환          4         3
Name: y_spot, Length: 

#### 회귀모델과 분류모델을 통한 학습 및 점수 확인

In [ ]:
X = df_szone[["pitcher_name", "hitter_name", "inning", "ball_type"]]
X_cate = X[["pitcher_name", "hitter_name", "ball_type"]]
X_conti = X.inning.to_frame()

y_x = df_szone[["x_spot"]]
y_y = df_szone[["y_spot"]]
y_btype = df_szone[["ball_type"]]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
ms = MinMaxScaler()
ohe = OneHotEncoder()

data = ohe.fit_transform(X_cate).toarray()
cols = ohe.get_feature_names_out()
s_X = pd.DataFrame(data, columns=cols)
X_cate = pd.DataFrame(data, columns=cols)

data = ms.fit_transform(X_conti)
cols = ms.get_feature_names_out()
X_conti = pd.DataFrame(data, columns=cols)

s_X = pd.concat([X_cate, X_conti], axis=1)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, train_test_split

tr_x, tt_x, tr_y, tt_y = train_test_split(s_X, y_x)

m = KNeighborsRegressor(n_neighbors=3)

m.fit(tr_x, tr_y)

m.score(tr_x, tr_y), m.score(tt_x, tt_y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split

C_m = KNeighborsClassifier()

tr_x, tt_x, tr_y, tt_y = train_test_split(s_X, y_btype)

C_m.fit(tr_x, tr_y)

C_m.score(tr_x, tr_y), C_m.score(tt_x, tt_y)
#print(cross_val_score(R_m, s_X, y_x))
#print(cross_val_score(R_m, s_X, y_y))
#print(cross_val_score(C_m, s_X, y_btype))

/Users/dokw/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


## 스트라이크존 분석 및 학습데이터 생성

### 데이터를 통해 확인할 수 있는 가설  
- 타자의 체중, 신장. 즉 타자마다 스트라이크 존의 위치와 크기가 다르다.
- 심판마다 스트라이크 존의 위치와 크기가 다르다.
- 이닝마다 심판의 스트라이크 존의 위치와 크기가 달라진다.
- 구종에 따라 심판의 스트라이크 존의 위치와 크기가 달라진다.

#### 타자마다 다른 스트라이크 존의 위치와 크기

In [ ]:
# 타자마다 다른 스트라이크 존의 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name"]).x_spot.median()}

중심점의 y좌표
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
hitter_height(cm)  hitter_weight(kg)  hitter_name
163                62                 김성윤            118.577414
                   64                 김지찬            117.980179
165                77                 김선빈             90.960785
168                73                 최경모             84.930819
170                73                 김태진            122.195877
                                                        ...    
190                85                 나승엽            109.485096
                   97                 윌리엄스           123.405053
                   99                 구드럼             90.449190
192                100                이재원             95.247084
194                85                 전사민             74.583199
Name: x_spot, Length: 328, dtype: float64

중심점의 y좌표
hitter_height(cm)  hitter_weight(kg)  hitter_name
163                62                 김성윤            148.401892
                   64                 김지찬            166.370029
165    

#### 투구 포지션에 따라 달라지는 타자별 스트라이크 존의 위치와 크기

In [ ]:
# 투구 포지션에 따라 다른 타자별 스트라이크 존의 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).x_spot.median()}

중심점의 y좌표
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
hitter_height(cm)  hitter_weight(kg)  hitter_name  pitcher_position
163                62                 김성윤          L                   122.011335
                                                   R                   116.634233
                   64                 김지찬          L                   124.431600
                                                   R                   115.037777
165                77                 김선빈          L                    95.517200
                                                                          ...    
190                99                 구드럼          L                    79.180336
                                                   R                    97.145314
192                100                이재원          L                   102.790651
                                                   R                    93.271876
194                85                 전사민          R                    74.583199
Name: x_spot, Length:

#### 심판마다 다른 스트라이크 존의 위치와 크기

In [ ]:
# 심판마다 다른 스트라이크 존의 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["refree"]).x_spot.median()}

중심점의 y좌표
{df_szone.groupby(["refree"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
refree
강광회    110.698670
구명환    102.493813
권영철    107.515561
김갑수    103.047070
김병주    105.134206
          ...    
최수원    104.263363
최영주    106.946794
추평호    105.561906
함지웅    104.611096
황인태     87.018929
Name: x_spot, Length: 40, dtype: float64

중심점의 y좌표
refree
강광회    152.779492
구명환    151.066669
권영철    150.635858
김갑수    155.571949
김병주    151.563492
          ...    
최수원    152.418166
최영주    151.467080
추평호    150.503835
함지웅    147.973686
황인태    144.770395
Name: y_spot, Length: 40, dtype: float64

중심점의 너비 (x좌표 최댓값 - 최솟값)
refree
강광회    172.035443
구명환    174.416329
권영철    200.333471
김갑수    161.120700
김병주    191.776786
          ...    
최수원    221.081286
최영주    157.759529
추평호    191.816400
함지웅    165.322500
황인태    156.288429
Name: x_spot, Length: 40, dtype: float64

중심점의 높이 (y좌표 최댓값 - 최솟값)
refree
강광회    222.553189
구명환    223.613714
권영철    221.357166
김갑수    192.087177
김병주    217.836846
          ...    
최수원    220.079497
최영주    207.672823
추평호    222.286537
함지웅    200.034606
황인태   

#### 타자마다 다른 심판 판정 스트라이크 존의 위치와 크기

In [ ]:
# 타자마다 다른 심판 판정 스트라이크 존의 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["refree", "hitter_name"]).x_spot.median()}
중심점의 y좌표
{df_szone.groupby(["refree", "hitter_name"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "hitter_name"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "hitter_name"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
refree  hitter_name
강광회     강민호             81.700811
        강백호            128.116870
        강승호             92.469989
        강진성            144.935283
        강한울            143.606391
                          ...    
황인태     정훈              56.072047
        채은성             93.990909
        최항             135.422147
        한동희             62.499969
        황재균             47.720617
Name: x_spot, Length: 8245, dtype: float64

중심점의 y좌표
refree  hitter_name
강광회     강민호            175.688063
        강백호            139.142052
        강승호            135.572223
        강진성            136.507675
        강한울            198.540178
                          ...    
황인태     정훈             137.411858
        채은성            132.692909
        최항             153.570760
        한동희            144.770395
        황재균            151.677275
Name: y_spot, Length: 8245, dtype: float64

중심점의 너비 (x좌표 최댓값 - 최솟값)
refree  hitter_name
강광회     강민호             90.989247
        강백호            141.6

#### 이닝마다 달라지는 심판의 스트라이크 존 위치와 크기

In [ ]:
# 이닝마다 달라지는 심판의 스트라이크 존 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["refree", "inning"]).x_spot.median()}

중심점의 y좌표
{df_szone.groupby(["refree", "inning"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "inning"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "inning"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
refree  inning
강광회     1         113.205381
        2         104.555501
        3         112.203409
        4         112.108871
        5         110.400421
                     ...    
황인태     5         103.683719
        6          64.249241
        7          55.157360
        8          69.856341
        9          91.436660
Name: x_spot, Length: 463, dtype: float64

중심점의 y좌표
refree  inning
강광회     1         155.545892
        2         147.752200
        3         151.411492
        4         158.072566
        5         154.966555
                     ...    
황인태     5         150.052178
        6         108.862783
        7         126.302829
        8         185.243218
        9         167.222098
Name: y_spot, Length: 463, dtype: float64

중심점의 너비 (x좌표 최댓값 - 최솟값)
refree  inning
강광회     1         162.699900
        2         158.499443
        3         153.168300
        4         148.714714
        5         158.317486
                     ...    
황인태     5      

#### 구종에 따라 달라지는 심판의 스트라이크 존의 위치와 크기

In [ ]:
#### 구종에 따라 달라지는 심판의 스트라이크 존의 위치와 크기
print(f"""중심점의 x좌표
{df_szone.groupby(["refree", "ball_type"]).x_spot.median()}

중심점의 y좌표
{df_szone.groupby(["refree", "ball_type"]).y_spot.median()}

중심점의 너비 (x좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "ball_type"]).x_spot.apply(lambda x : max(x) - min(x))}

중심점의 높이 (y좌표 최댓값 - 최솟값)
{df_szone.groupby(["refree", "ball_type"]).y_spot.apply(lambda x : max(x) - min(x))}""")

중심점의 x좌표
refree  ball_type
강광회     너클            88.632909
        슬라이더         112.464191
        직구           109.098119
        체인지업         109.827491
        커브           109.190373
                        ...    
황인태     직구            91.625869
        체인지업          75.340704
        커브            75.742621
        투심            43.316919
        포크            35.435086
Name: x_spot, Length: 291, dtype: float64

중심점의 y좌표
refree  ball_type
강광회     너클            92.904520
        슬라이더         145.979446
        직구           156.946898
        체인지업         151.484452
        커브           149.757732
                        ...    
황인태     직구           149.577938
        체인지업         151.677275
        커브           108.862783
        투심           132.835355
        포크            99.330212
Name: y_spot, Length: 291, dtype: float64

중심점의 너비 (x좌표 최댓값 - 최솟값)
refree  ball_type
강광회     너클             0.000000
        슬라이더         162.641486
        직구           171.560071
        체인지업      

### 서비스 제공을 위한 값 선정
1. 타자 신장, 타자 몸무게, 타자명 피쳐를 기준으로 산출된 타자별 스트라이크 존의 중심 x, y 값과 너비와 높이
2. 심판명과 타자명 피쳐를 기준으로 산출된 심판별 스트라이크 존의 중심 x, y 값과 너비와 높이
    - 이닝 피쳐의 데이터 변화에 따라 달라지는 스트라이크 존 구성요소의 오차값 가감
    - 구종 피쳐의 데이터 변화에 따라 달라지는 스트라이크 존 구성요소의 오차값 가감
3. 구종 피쳐의 데이터 중 오차값이 가장 큰 구종의 정보 제공

### 오차값 산출
- Default 로 심판명 피쳐만으로 구해진 스트라이크 존의 중심 x, y값과 너비와 높이를 사용한다.
- Default 와 아래 값과의 오차값을 계산한다.
	- 심판명과 현재 이닝번호를 기준으로 산출된 스트라이크 존의 x, y값과 너비와 높이
	- 심판명과 구종을 기준으로 산출된 스트라이크 존의 x, y값과 너비와 높이

### 타자별 스트라이크존 학습 데이터 구성
- 학습 대상 피쳐: "hitter_height(cm)", "hitter_weight(kg)", "hitter_name"
#### m_hitter_szone_nx 학습용 데이터
    - 라벨데이터(y): hitter_szone_nx / x_spot 중앙값
#### m_hitter_szone_ny 학습용 데이터
    - 라벨데이터(y): hitter_szone_ny / y_spot 중앙값
#### m_hitter_szone_width 학습용 데이터
    - 라벨데이터(y): hitter_szone_height / x_spot.max() - x_spot.min()
#### m_hitter_szone_height 학습옹 데이터
    - 라벨데이터(y): hitter_szone_height / y_spot.max() - y_spot.min()


In [ ]:
df_szone["hitter_szone_nx"] = df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).x_spot.transform(np.median)

In [ ]:
df_szone["hitter_szone_ny"] = df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).y_spot.transform(np.median)

In [ ]:
df_szone["hitter_szone_width"] = df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).x_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone["hitter_szone_height"] = df_szone.groupby(["hitter_height(cm)", "hitter_weight(kg)", "hitter_name", "pitcher_position"]).y_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone.drop_duplicates(inplace=True)
df_szone.reset_index(drop=True, inplace=True)

In [ ]:
drop_idx = df_szone[(df_szone.hitter_szone_nx == 0) | (df_szone.hitter_szone_ny == 0) | (df_szone.hitter_szone_width == 0) | (df_szone.hitter_szone_height == 0)].index
df_szone.drop(drop_idx, inplace=True)

### 심판시점 타자별 스트라이크존 구성
- 학습 대상 피쳐: "refree", "hitter_name"
#### m_refree_szone_hitter_nx 학습용 데이터
    - 라벨데이터(y): refree_szone_hitter_nx / x_spot 중앙값
#### m_refree_szone_hitter_ny 학습용 데이터
    - 라벨데이터(y): refree_szone_hitter_ny / y_spot 중앙값
#### m_refree_szone_hitter_width 학습용 데이터
    - 라벨데이터(y): refree_szone_hitter_width / x_spot.max() - x_spot.min()
#### m_refree_szone_hitter_height 학습용 데이터
    - 라벨데이터(y): refree_szone_hitter_height / y_spot.max() - y_spot.min()

In [ ]:
df_szone["refree_szone_hitter_nx"] = df_szone.groupby(["refree", "hitter_name"]).x_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_hitter_ny"] = df_szone.groupby(["refree", "hitter_name"]).y_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_hitter_width"] = df_szone.groupby(["refree", "hitter_name"]).x_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone["refree_szone_hitter_height"] = df_szone.groupby(["refree", "hitter_name"]).y_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone.drop_duplicates(inplace=True)
df_szone.reset_index(drop=True, inplace=True)

In [ ]:
drop_idx = df_szone[(df_szone.refree_szone_hitter_nx == 0) | (df_szone.refree_szone_hitter_ny == 0) | (df_szone.refree_szone_hitter_width == 0) | (df_szone.refree_szone_hitter_height == 0)].index
df_szone.drop(drop_idx, inplace=True)

### 심판 스트라이크존 오차값 산출을 위한 기본값 구성
- 학습 대상 피쳐: "refree"
#### m_refree_szone_default_nx 학습용 데이터
    - 라벨데이터(y): refree_szone_default_nx / x_spot 중앙값
#### m_refree_szone_default_ny  학습용 데이터
    - 라벨데이터(y): refree_szone_default_ny / y_spot 중앙값
#### m_refree_szone_default_width 학습용 데이터
    - 라벨데이터(y): refree_szone_default_width / y_spot.max() - y_spot.min()
#### m_refree_szone_default_height 학습용 데이터
    - 라벨데이터(y): refree_szone_default_height / y_spot.max() - y_spot.min()

In [ ]:
df_szone["refree_szone_default_nx"] = df_szone.groupby(["refree"]).x_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_default_ny"] = df_szone.groupby(["refree"]).y_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_default_width"] = df_szone.groupby(["refree"]).x_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone["refree_szone_default_height"] = df_szone.groupby(["refree"]).y_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone.drop_duplicates(inplace=True)
df_szone.reset_index(drop=True, inplace=True)

In [ ]:
drop_idx = df_szone[(df_szone.refree_szone_default_nx == 0) | (df_szone.refree_szone_default_ny == 0) | (df_szone.refree_szone_default_width == 0) | (df_szone.refree_szone_default_height == 0)].index
df_szone.drop(drop_idx, inplace=True)

### 심판 스트라이크존 오차값 산출을 위한 이닝별 스트라이크존 구성
- 학습 대상 피쳐: "refree", "inning"
#### m_refree_szone_inning_nx 학습용 데이터
    - 라벨데이터(y): refree_szone_inning_nx / x_spot 중앙값
#### m_refree_szone_inning_ny  학습용 데이터
    - 라벨데이터(y): refree_szone_inning_ny / y_spot 중앙값
#### m_refree_szone_inning_width 학습용 데이터
    - 라벨데이터(y): refree_szone_inning_width / y_spot.max() - y_spot.min()
#### m_refree_szone_inning_height 학습용 데이터
    - 라벨데이터(y): refree_szone_inning_height / y_spot.max() - y_spot.min()

In [ ]:
df_szone["refree_szone_inning_nx"] = df_szone.groupby(["refree", "inning"]).x_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_inning_ny"] = df_szone.groupby(["refree", "inning"]).y_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_inning_width"] = df_szone.groupby(["refree", "inning"]).x_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone["refree_szone_inning_height"] = df_szone.groupby(["refree", "inning"]).y_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone.drop_duplicates(inplace=True)
df_szone.reset_index(drop=True, inplace=True)

In [ ]:
drop_idx = df_szone[(df_szone.refree_szone_inning_nx == 0) | (df_szone.refree_szone_inning_ny == 0) | (df_szone.refree_szone_inning_width == 0) | (df_szone.refree_szone_inning_height == 0)].index
df_szone.drop(drop_idx, inplace=True)

### 심판 스트라이크존 오차값 산출을 위한 구종별 스트라이크존 구성
- 학습 대상 피쳐: "refree", "ball_type"
#### m_refree_szone_btype_nx 학습용 데이터
    - 라벨데이터(y): refree_szone_btype_nx / x_spot 중앙값
#### m_refree_szone_btype_ny  학습용 데이터
    - 라벨데이터(y): refree_szone_btype_ny / y_spot 중앙값
#### m_refree_szone_btype_width 학습용 데이터
    - 라벨데이터(y): refree_szone_btype_width / y_spot.max() - y_spot.min()
#### m_refree_szone_btype_height 학습용 데이터
    - 라벨데이터(y): refree_szone_btype_height / y_spot.max() - y_spot.min()

In [ ]:
df_szone["refree_szone_btype_nx"] = df_szone.groupby(["refree", "ball_type"]).x_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_btype_ny"] = df_szone.groupby(["refree", "ball_type"]).y_spot.transform(np.median)

In [ ]:
df_szone["refree_szone_btype_width"] = df_szone.groupby(["refree", "ball_type"]).x_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone["refree_szone_btype_height"] = df_szone.groupby(["refree", "ball_type"]).y_spot.transform(lambda x : max(x)-min(x))

In [ ]:
df_szone.drop_duplicates(inplace=True)
df_szone.reset_index(drop=True, inplace=True)

In [ ]:
drop_idx = df_szone[(df_szone.refree_szone_btype_nx == 0) | (df_szone.refree_szone_btype_ny == 0) | (df_szone.refree_szone_btype_width == 0) | (df_szone.refree_szone_btype_height == 0)].index
df_szone.drop(drop_idx, inplace=True)

In [ ]:
df_szone.to_csv("S존 학습용 데이터.csv", index=False)

In [ ]:
df_szone["hitter_height(cm)"]

0         183
1         183
2         183
3         183
4         183
         ... 
127682    188
127683    188
127684    188
127685    188
127686    188
Name: hitter_height(cm), Length: 127678, dtype: object

## 심판선택을 위한 심판정보 JSON 생성
refree 이름만 추출

In [ ]:
refrees = pd.read_csv("KBO전략데이터_서비스용.csv")

In [ ]:
refrees = refrees.refree.to_frame()

In [ ]:
refrees.drop_duplicates().reset_index(drop=True).to_json("KBO심판정보.json")

## 2019-2023 KBO 전략데이터 Join

In [ ]:
import pandas as pd

In [ ]:
p_all = pd.read_csv("투구정보 통합 - 정품.csv").dropna()
game_all = pd.read_csv("경기정보 통합.csv").dropna()
hitter_all = pd.read_csv("타자정보 통합.csv").dropna()

In [ ]:
pgame = pd.merge(p_all, game_all, on = ["game_date", "home_name", "away_name"])

In [ ]:
total = pd.merge(pgame, hitter_all.drop("hitter_position", axis=1), on = ["hitter_name"])

In [ ]:
total.to_csv("KBO전략데이터.csv", index=False)

## 2019~2023 투수정보 Join

In [ ]:
p_2019 = pd.read_csv("투수정보_2019.csv")
p_2020 = pd.read_csv("투수정보_2020.csv")
p_2021 = pd.read_csv("투수정보_2021.csv")
p_2022 = pd.read_csv("투수정보_2022.csv")
p_2023 = pd.read_csv("투수정보_2023.csv")

In [ ]:
final = pd.concat([p_2019, p_2020, p_2021, p_2022, p_2023])

In [ ]:
final.to_csv("2019 투구정보.csv", index=False)

In [ ]:
final

,game_date,home_name,away_name,inning,x_spot,y_spot,ball_speed,judgement,ball_type,pitcher_name,pitcher_potision,hitter_name,hitter_position,refree
0,2021-09-14,두산,KT,1,60.214627,62.269138,144,볼,직구,데스파이네,R,김인태,L,권영철
1,2021-09-14,두산,KT,1,86.564841,193.034303,147,스트라이크,직구,데스파이네,R,김인태,L,권영철
2,2021-09-14,두산,KT,1,222.570286,117.047332,147,볼,투심,데스파이네,R,김인태,L,권영철
3,2021-09-14,두산,KT,1,127.595707,64.039286,148,헛스윙,직구,데스파이네,R,김인태,L,권영철
4,2021-09-14,두산,KT,1,135.130210,140.988635,117,파울,체인지업,데스파이네,R,김인태,L,권영철
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31669,2021-10-09,SSG,롯데,9,115.847990,105.697709,145,헛스윙 삼진,직구,김택형,L,이대호,R,김익수
31670,2021-10-09,SSG,롯데,9,32.807786,-12.000000,146,볼,직구,김택형,L,전준우,R,김익수
31671,2021-10-09,SSG,롯데,9,51.933429,88.209023,144,볼,직구,김택형,L,전준우,R,김익수
31672,2021-10-09,SSG,롯데,9,67.813520,148.850943,145,스트라이크,직구,김택형,L,전준우,R,김익수


## 2019 ~ 2023 투구정보 Join

In [ ]:
p_2019 = pd.read_csv("2019 투구정보 - 정품.csv")
p_2020 = pd.read_csv("2020 투구정보 - 정품.csv")
p_2021 = pd.read_csv("2021 투구정보 - 정품.csv")
p_2022 = pd.read_csv("2022 투구정보 - 정품.csv")
p_2023 = pd.read_csv("2023 투구정보 - 정품.csv")

In [ ]:
final = pd.concat([p_2019, p_2020, p_2021, p_2022, p_2023])

In [ ]:
final.to_csv("투구정보 통합 - 정품.csv", index=False)

In [ ]:
final

,game_date,refree,home_name,away_name,inning,x_spot,y_spot,ball_speed,judgement,ball_type,pitcher_name,pitcher_potision,hitter_name,hitter_position
0,2019-03-23,강광회,두산,한화,1,38.360836,194.344109,136.0,헛스윙,슬라이더,린드블럼,R,호잉,L
1,2019-03-23,강광회,두산,한화,1,160.930457,157.847606,143.0,스트라이크,투심,린드블럼,R,호잉,L
2,2019-03-23,강광회,두산,한화,1,7.345871,292.000000,130.0,볼,포크,린드블럼,R,호잉,L
3,2019-03-23,강광회,두산,한화,1,223.000000,44.360063,144.0,볼,직구,린드블럼,R,호잉,L
4,2019-03-23,강광회,두산,한화,1,132.259719,115.644589,138.0,1루타,커터,린드블럼,R,호잉,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140384,2023-08-09,유덕형,키움,롯데,9,120.143253,258.018212,141.0,볼,직구,주승우,R,이정훈,L
140385,2023-08-09,유덕형,키움,롯데,9,91.171647,-12.000000,139.0,볼,직구,주승우,R,이정훈,L
140386,2023-08-09,유덕형,키움,롯데,9,69.761334,142.410486,120.0,스트라이크,슬라이더,이명종,R,안치홍,R
140387,2023-08-09,유덕형,키움,롯데,9,177.989443,50.602486,124.0,볼,슬라이더,이명종,R,안치홍,R
